In [2]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df=pd.read_csv('./new_train.csv')
# 
df[515:650]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
515,3636015289,57,272.0,12849827,87,724.0,4060339532,34998,GPD,3,0,12,0,0,0
516,3636015289,58,280.0,12849827,88,732.0,2591107590,34998,GPD,3,0,12,0,0,0
517,3636015289,59,286.0,12849827,89,738.0,1384712649,34998,GPD,3,0,12,0,0,0
518,3636015289,60,298.0,12849827,90,750.0,1528528240,34998,GPD,3,0,12,0,0,0
519,3636015289,61,300.0,12849827,91,752.0,4044602144,34998,GPD,3,0,12,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,839643180,120,568.0,12849827,217,1590.0,3295148391,34998,GPD,0,0,4,0,0,1
646,839643180,121,572.0,12849827,218,1594.0,3536587712,34998,GPD,0,0,4,0,0,1
647,839643180,122,576.0,12849827,219,1598.0,1537073604,34998,GPD,0,0,4,0,0,1
648,4233815620,0,0.0,12849827,220,1988.0,3914505153,34998,GPD,3,0,8,0,0,2


***Creating subid parquet files***


In [14]:
import pandas as pd
import os

metadata = pd.read_csv('./train.csv')

# Define the sampling rate (200 Hz = 200 samples per second)
sampling_rate = 200

# Defining the duration of the window to extract (50 seconds)
window_duration = 50 * sampling_rate  

output_dir = 'testingFiles/'
os.makedirs(output_dir, exist_ok=True)

files_per_category = 1000

grouped_metadata = metadata.groupby('expert_consensus')

for expert_consensus, group in grouped_metadata:
    print(f"Processing category {expert_consensus}")
    
    category_files = group.head(files_per_category)
    
    for i, row in category_files.iterrows():
        if i % 100 == 0:  
            print(f"Processing row {i} for category {expert_consensus}")

        eeg_id = row['eeg_id']
        eeg_sub_id = row['eeg_sub_id']
        offset_seconds = row['eeg_label_offset_seconds']
        eeg_data = pd.read_parquet(f'./train_eegs/{eeg_id}.parquet')

        offset_samples = int(offset_seconds * sampling_rate)

        # Extract the relevant portion of the EEG data (50 seconds of data)
        eeg_subsample = eeg_data.iloc[offset_samples:offset_samples + window_duration]

        output_file = f'{output_dir}/{eeg_id}_{eeg_sub_id}.parquet'

        # Saving subsample as a new .parquet file
        eeg_subsample.to_parquet(output_file, index=False)

print(f"Processed and saved 1000 files per expert_consensus category.")


Processing category GPD
Processing row 500 for category GPD
Processing row 600 for category GPD
Processing row 700 for category GPD
Processing row 800 for category GPD
Processing row 1000 for category GPD
Processing row 2000 for category GPD
Processing row 2300 for category GPD
Processing row 2900 for category GPD
Processing row 3600 for category GPD
Processing row 3700 for category GPD
Processing category GRDA
Processing row 300 for category GRDA
Processing row 1300 for category GRDA
Processing row 2200 for category GRDA
Processing row 2700 for category GRDA
Processing row 2800 for category GRDA
Processing row 3400 for category GRDA
Processing row 4000 for category GRDA
Processing row 4500 for category GRDA
Processing row 4600 for category GRDA
Processing row 4700 for category GRDA
Processing category LPD
Processing row 1100 for category LPD
Processing row 1700 for category LPD
Processing row 1800 for category LPD
Processing row 3300 for category LPD
Processing row 3800 for category L

***File to generate train.csv file for the selected data***

In [4]:
import pandas as pd
import os

metadata = pd.read_csv('./train.csv')

file_dir = './testingFiles/'

filtered_rows = []

# Looping through each row in the metadata and check if the corresponding file exists
for i, row in metadata.iterrows():
    eeg_id = row['eeg_id']
    eeg_sub_id = row['eeg_sub_id']

    file_path = f'{file_dir}/{eeg_id}_{eeg_sub_id}.parquet'
    
    if os.path.exists(file_path):
        # If the file exists, add the row to the filtered list
        filtered_rows.append(row)

filtered_metadata = pd.DataFrame(filtered_rows)

filtered_metadata.to_csv('new_train.csv', index=False)

print(f"New train.csv created with {len(filtered_metadata)} entries based on available files.")


New train.csv created with 6000 entries based on available files.


In [5]:
import pandas as pd

X_master = pd.DataFrame()  # To store features (EEG subsamples)
y_master = pd.DataFrame()  # To store labels (votes)
chunk_size = 1000  
num_files_to_save = 6000  

metadata = pd.read_csv('./new_train.csv')

# Loop through the metadata to extract features and labels
for i, row in metadata.iterrows():
    if i >= num_files_to_save:
        break  
    
    if i % 500 == 0: 
        print("Processing row", i)
    
    eeg_id = row['eeg_id']
    eeg_sub_id = row['eeg_sub_id']

    # Loading corresponding EEG data from parquet
    eeg_data = pd.read_parquet(f'./testingFiles/{eeg_id}_{eeg_sub_id}.parquet')

    # Flatten the EEG data and append to X
    X_chunk = pd.DataFrame(eeg_data.values.flatten()).transpose()  # Flatten to 1D and ensure it's a row

    # Append corresponding label to y
    y_chunk = pd.DataFrame([row['expert_consensus']], columns=['expert_consensus'])

    # Concatenate the current chunk to the master DataFrame
    X_master = pd.concat([X_master, X_chunk], ignore_index=True)
    y_master = pd.concat([y_master, y_chunk], ignore_index=True)

    # Clear the current chunk after processing (optional, as we don't need to store chunks)
    X_chunk = None
    y_chunk = None

print("Processing complete. All data loaded into X_master and y_master.")


X=X_master
y=y_master
print(X.shape)
print(y.shape)

Processing row 0
Processing row 199
Processing row 398
Processing row 597
Processing row 796
Processing row 995
Processing row 1194
Processing row 1393
Processing row 1592
Processing row 1791
Processing row 1990
Processing row 2189
Processing row 2388
Processing row 2587
Processing row 2786
Processing row 2985
Processing row 3184
Processing row 3383
Processing row 3582
Processing row 3781
Processing row 3980
Processing row 4179
Processing row 4378
Processing row 4577
Processing row 4776
Processing row 4975
Processing row 5174
Processing row 5373
Processing row 5572
Processing row 5771
Processing row 5970
Processing complete. All data loaded into X_master and y_master.


In [4]:
import pickle
import os

file_X = 'X_data.pkl'
file_y = 'y_data.pkl'

# Check if the files already exist
if not os.path.exists(file_X) or not os.path.exists(file_y):
    # Save X and y to pickle files if they don't exist
    with open(file_X, 'wb') as f:
        pickle.dump(X_master, f)
    with open(file_y, 'wb') as f:
        pickle.dump(y_master, f)
    print(f"Data has been saved to {file_X} and {file_y}")
else:
    # If the files exist, load X and y from the pickle files
    with open(file_X, 'rb') as f:
        X = pickle.load(f)
    with open(file_y, 'rb') as f:
        y = pickle.load(f)
    print("Data has been loaded from existing pickle files.")




Data has been loaded from existing pickle files.


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import pickle
import os


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # This will transform string labels to numerical values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


model_file = 'model.pkl'

# Check if the model file already exists
if not os.path.exists(model_file):
    
    # Training the Random Forest Classifier
    forest = RandomForestClassifier(n_estimators=500,max_depth=10,min_samples_leaf=2,min_samples_split=13,max_features='sqrt', random_state=42)
    forest.fit(X_train, y_train)

    # Saving  model to a pickle file if it doesn't exist
    with open(model_file, 'wb') as f:
        pickle.dump(forest, f)
    print(f"Model has been saved to {model_file}")
else:
    # If the model file exists, load the model from the pickle file
    with open(model_file, 'rb') as f:
        forest = pickle.load(f)
    print("Model has been loaded from existing pickle file.")




# Predict on testing set
y_pred = forest.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"test Accuracy: {accuracy:.4f}")

# Print classification report for detailed metrics
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


c:\Users\mridu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model has been loaded from existing pickle file.
test Accuracy: 0.5933
              precision    recall  f1-score   support

         GPD       0.77      0.70      0.73       214
        GRDA       0.59      0.68      0.63       181
         LPD       0.46      0.67      0.55       188
        LRDA       0.79      0.63      0.70       223
       Other       0.53      0.31      0.39       201
     Seizure       0.48      0.57      0.52       193

    accuracy                           0.59      1200
   macro avg       0.60      0.59      0.59      1200
weighted avg       0.61      0.59      0.59      1200



In [6]:

yy=pd.DataFrame(y_train)

value_counts = yy.value_counts()

# Print the counts
print(value_counts)


yy=pd.DataFrame(y_test)

value_counts = yy.value_counts()

# Print the counts
print(value_counts)


0
1    819
2    812
5    807
4    799
0    786
3    777
Name: count, dtype: int64
0
3    223
0    214
4    201
5    193
2    188
1    181
Name: count, dtype: int64
